In [1]:
# To show the output of all lines in a cell rather that just the last line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd

In [3]:
data_amazon = pd.read_table("~/Downloads/sentiment_labelled_sentences/amazon_cells_labelled.txt"
                          , sep='\t', header=None, names=['sentence', 'sentiment'])
data_imdb = pd.read_table("~/Downloads/sentiment_labelled_sentences/imdb_labelled.txt"
                          , sep='\t', header=None, names=['sentence', 'sentiment'])
data_yelp = pd.read_table("~/Downloads/sentiment_labelled_sentences/yelp_labelled.txt"
                          , sep='\t', header=None, names=['sentence', 'sentiment'])

In [4]:
data_amazon.head(3)
data_imdb.head(3)
data_yelp.head(3)

,sentence,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1


,sentence,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0


,sentence,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0


In [5]:
data_amazon.shape
data_imdb.shape
data_yelp.shape

(1000, 2)

(748, 2)

(1000, 2)

In [7]:
all_data = pd.concat([data_amazon, data_imdb, data_yelp])

In [8]:
all_data.shape

(2748, 2)

In [9]:
all_data.head(3)
all_data.shape

,sentence,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1


(2748, 2)

In [205]:
sample_data = [['Good case for the money!', 1], ['Do not waste your money.', 0], ['Good product. Love it!', 1]] 
small_sample = pd.DataFrame(sample_data, columns = ['sentence', 'sentiment']) 

In [206]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [207]:
lr = LogisticRegression(penalty='l2', C=.8, random_state=21)

In [208]:
# ??CountVectorizer
# ??TfidfTransformer
# ??LogisticRegression

In [209]:
count_vec = CountVectorizer(binary=False, stop_words='english', ngram_range=(1,1))
# count_vec = CountVectorizer(binary=False, stop_words='english', ngram_range=(2,2))
# count_vec = CountVectorizer(binary=False, stop_words='None', ngram_range=(2,2))
count_vec.fit(small_sample.sentence)
small_transformed = count_vec.transform(small_sample.sentence)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [210]:
pd.set_option('display.max_colwidth', -1)
small_sample

,sentence,sentiment
0,Good case for the money!,1
1,Do not waste your money.,0
2,Good product. Love it!,1


In [211]:
from pandas import DataFrame

print(DataFrame(small_transformed.A, columns=count_vec.get_feature_names()).to_string())

   case  good  love  money  product  waste
0  1     1     0     1      0        0    
1  0     0     0     1      0        1    
2  0     1     1     0      1        0    


In [212]:
type(small_transformed)
small_transformed.A # .A return an ndarray from the sparse matrix

scipy.sparse.csr.csr_matrix

array([[1, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 1],
       [0, 1, 1, 0, 1, 0]])

In [213]:
tfidf = TfidfTransformer(use_idf=True)
tfidf.fit(small_transformed)
small_tfidfed = tfidf.transform(small_transformed)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [214]:
print(DataFrame(small_tfidfed.A, columns=count_vec.get_feature_names()).to_string())

       case      good      love     money   product     waste
0  0.680919  0.517856  0.000000  0.517856  0.000000  0.000000
1  0.000000  0.000000  0.000000  0.605349  0.000000  0.795961
2  0.000000  0.473630  0.622766  0.000000  0.622766  0.000000


In [215]:
small_tfidfed.A

array([[ 0.68091856,  0.51785612,  0.        ,  0.51785612,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.60534851,  0.        ,
         0.79596054],
       [ 0.        ,  0.4736296 ,  0.62276601,  0.        ,  0.62276601,
         0.        ]])

In [199]:
text_classifier = Pipeline([
    ('vectorizer', CountVectorizer(binary=False, stop_words='english', ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', lr),
#     ('clf', SGDClassifier()),
])

In [87]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
     all_data.sentence, all_data.sentiment, test_size=0.35, random_state=4)

In [88]:
text_classifier.fit(X_train, y_train) 

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
  ...alty='l2', random_state=21, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [89]:
X_test[0:4]
y_test[0:4]

177    The atmosphere is modern and hip, while mainta...
720                        Cute, quaint, simple, honest.
525            It's an empty, hollow shell of a movie.  
630                      Don't bother - go to the store.
Name: sentence, dtype: object

177    1
720    1
525    0
630    0
Name: sentiment, dtype: int64

In [90]:
type(X_test)
type(y_test)

pandas.core.series.Series

pandas.core.series.Series

In [91]:
X_test = X_test.reset_index(drop = True) # drop=True discards the old index
X_test[0:4]

y_test = y_test.reset_index(drop = True)
y_test[0:4]

0    The atmosphere is modern and hip, while mainta...
1                        Cute, quaint, simple, honest.
2            It's an empty, hollow shell of a movie.  
3                      Don't bother - go to the store.
Name: sentence, dtype: object

0    1
1    1
2    0
3    0
Name: sentiment, dtype: int64

In [92]:
predicted_test = text_classifier.predict(X_test)
predicted_proba_test = text_classifier.predict_proba(X_test)

for i in range(5):
    print("{}, {}, {}, {}".format(X_test[i], predicted_test[i], predicted_proba_test[i], y_test[i]))

The atmosphere is modern and hip, while maintaining a touch of coziness., 1, [ 0.48580128  0.51419872], 1
Cute, quaint, simple, honest., 0, [ 0.55741558  0.44258442], 1
It's an empty, hollow shell of a movie.  , 1, [ 0.44886372  0.55113628], 0
Don't bother - go to the store., 0, [ 0.65667888  0.34332112], 0
To those who find this movie intelligent or even masterful, I can only say - it's your intelligence and your imagination you obviously used to try and make some sense of this pitiful attempt (it's in our human nature to try and make sense of things) .  , 0, [ 0.5411766  0.4588234], 0


In [93]:
from sklearn import metrics

y_train = y_train.astype('category')
predicted_train = text_classifier.predict(X_train)

print(metrics.classification_report(y_train, predicted_train,
    labels=y_train.cat.categories.tolist()))

metrics.confusion_matrix(y_train, predicted_train)

y_test = y_test.astype('category')
predicted_test = text_classifier.predict(X_test)

print(metrics.classification_report(y_test, predicted_test,
    labels=y_test.cat.categories.tolist()))

metrics.confusion_matrix(y_test, predicted_test)

             precision    recall  f1-score   support

          0       0.98      0.97      0.98       880
          1       0.97      0.98      0.98       906

avg / total       0.98      0.98      0.98      1786



array([[855,  25],
       [ 14, 892]])

             precision    recall  f1-score   support

          0       0.81      0.80      0.80       482
          1       0.80      0.81      0.80       480

avg / total       0.80      0.80      0.80       962



array([[386,  96],
       [ 93, 387]])

In [112]:
# ??metrics.classification_report(y_train, predicted_train, labels=y_train.cat.categories.tolist())
# ??metrics.precision_score
metrics.precision_score(y_test, predicted_test, average='macro') 
metrics.precision_score(y_test, predicted_test, average='micro') 
metrics.precision_score(y_test, predicted_test, average='weighted') 
metrics.recall_score(y_test, predicted_test, average='micro')

0.80354387375354963

0.80353430353430355

0.80354865886317273

0.80353430353430355

In [113]:
# Be aware of the macro vs. micro vs. weighted averages. Look at the one based on your problem. 
# In multi-class classification sometimes some of these metrics are equal. 
# E.g. micro-average precision and recall are always the same.

Trained model predicting on a single sample

In [95]:
text_classifier.predict_proba(['Dogs love us!'])

array([[ 0.27486995,  0.72513005]])

## One week later!!!

In [110]:
text_classification2 = text_classifier.fit(all_data.sentence, all_data.sentiment) 

In [111]:
from sklearn import metrics

y_train = y_train.astype('category')
predicted_train = text_classification2.predict(X_train)

print(metrics.classification_report(y_train, predicted_train,
    labels=y_train.cat.categories.tolist()))

metrics.confusion_matrix(y_train, predicted_train)

             precision    recall  f1-score   support

          0       0.97      0.97      0.97       880
          1       0.97      0.98      0.97       906

avg / total       0.97      0.97      0.97      1786



array([[850,  30],
       [ 22, 884]])

#### The model is now portable:

In [83]:
import pickle

# save the model to disk
filename = 'model_v1.pkl'
pickle.dump(text_clf2, open(filename, 'wb'), protocol=2)
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
print("%2.1f%% Positive" % (100 * loaded_model.predict_proba(["But he does like you!"])[0:1][:,1][0]))

42.5% Positive


In [85]:
"%2.1f%% Positive" % (100 * loaded_model.predict_proba(["But he does like you!"])[0:1][:,1][0])

'42.5% Positive'